In [1]:
from utils import sqlite_interface as sql
import numpy as np
import pandas as pd

In [2]:
DB_FILEPATH = '../data/sqlite/round_1_hs_pitchers.db'

In [3]:
raw_table_name = 'Raw'
sql.create_csv_table('../data/raw/round-1-hs-pitchers-drafted.csv', DB_FILEPATH, raw_table_name)

CSV file '../data/raw/round-1-hs-pitchers-drafted.csv' located successfully.
Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Data loaded into table 'Raw' successfully.
First 5 rows of 'Raw':
(2025, '1', None, 6, 'FrRnd', 6, 'Pirates', 'Y', '$7250000', 'Seth Hernandez\xa0(minors)', 'P', None, None, None, None, None, None, None, None, None, None, None, None, 'HS', 'Corona HS (Corona CA)', '-9999')
(2025, '1', None, 25, 'FrRnd', 25, 'Padres', 'Y', '$3606600', 'Kruz Schoolcraft\xa0(minors)', 'P', None, None, None, None, None, None, None, None, None, None, None, None, 'HS', 'Sunset HS (Portland OR)', '-9999')
(2024, '1', None, 24, 'FrRnd', 24, 'Braves', 'Y', '$3553800', 'Cam Caminiti\xa0(minors)', 'P', None, None, None, None, None, None, None, None, None, None, None, None, 'HS', 'Saguaro HS (Scottsdale AZ)', '-9999')
(2024, '1', None, 25, 'FrRnd', 25, 'Padres', 'Y', '$3442100', 'Kash Mayfield\xa0(minors)', 'P', None, None, None, None, None, None, None, None, None, N

In [4]:
sql.query_db(f"PRAGMA table_info({raw_table_name})", DB_FILEPATH)

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Query returned 26 results.


[(0, 'Year', 'INTEGER', 0, None, 0),
 (1, 'Rnd', 'TEXT', 0, None, 0),
 (2, 'DT', 'REAL', 0, None, 0),
 (3, 'OvPck', 'INTEGER', 0, None, 0),
 (4, 'FrRnd', 'TEXT', 0, None, 0),
 (5, 'RdPck', 'INTEGER', 0, None, 0),
 (6, 'Tm', 'TEXT', 0, None, 0),
 (7, 'Signed', 'TEXT', 0, None, 0),
 (8, 'Bonus', 'TEXT', 0, None, 0),
 (9, 'Name', 'TEXT', 0, None, 0),
 (10, 'Pos', 'TEXT', 0, None, 0),
 (11, 'WAR', 'REAL', 0, None, 0),
 (12, 'G', 'REAL', 0, None, 0),
 (13, 'AB', 'REAL', 0, None, 0),
 (14, 'HR', 'REAL', 0, None, 0),
 (15, 'BA', 'REAL', 0, None, 0),
 (16, 'OPS', 'REAL', 0, None, 0),
 (17, 'G.1', 'REAL', 0, None, 0),
 (18, 'W', 'REAL', 0, None, 0),
 (19, 'L', 'REAL', 0, None, 0),
 (20, 'ERA', 'REAL', 0, None, 0),
 (21, 'WHIP', 'REAL', 0, None, 0),
 (22, 'SV', 'REAL', 0, None, 0),
 (23, 'Type', 'TEXT', 0, None, 0),
 (24, 'Drafted Out of', 'TEXT', 0, None, 0),
 (25, 'Name-additional', 'TEXT', 0, None, 0)]

In [5]:
sql_filter_for_signed = f"SELECT year, name, ovpck, tm, bonus, war FROM {raw_table_name} WHERE signed='Y';"
signed_hs_pitchers_round_1 = sql.query_db(sql_filter_for_signed, DB_FILEPATH)
signed_hs_pitchers_round_1[0]

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Query returned 418 results.


(2025, 'Seth Hernandez\xa0(minors)', 6, 'Pirates', '$7250000', None)

In [6]:
df = pd.DataFrame(signed_hs_pitchers_round_1)
df

,0,1,2,3,4,5
0,2025,Seth Hernandez (minors),6,Pirates,$7250000,NaN
1,2025,Kruz Schoolcraft (minors),25,Padres,$3606600,NaN
2,2024,Cam Caminiti (minors),24,Braves,$3553800,NaN
3,2024,Kash Mayfield (minors),25,Padres,$3442100,NaN
4,2024,Braylon Doughty (minors),36,Guardians,$2569200,NaN
...,...,...,...,...,...,...
413,1965,Les Rohr (minors),2,Mets,$50000,-0.3
414,1965,Joe Coleman (minors),3,Senators,$65000,23.2
415,1965,Rick James (minors),6,Cubs,$40000,-0.5
416,1965,Scott McDonald (minors),15,Orioles,$30000,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       418 non-null    int64  
 1   1       418 non-null    object 
 2   2       418 non-null    int64  
 3   3       418 non-null    object 
 4   4       399 non-null    object 
 5   5       252 non-null    float64
dtypes: float64(1), int64(2), object(3)
memory usage: 19.7+ KB


In [8]:
df['year'] = df[0]
df['pick_number'] = df[2]
df['team'] = df[3]
df['player_name'] = df[1].apply(lambda x: x.removesuffix('(minors)').rstrip())
df['bonus'] = df[4].apply(lambda x: float(x.removeprefix('$')) if isinstance(x, str) else np.nan)
df['career_war'] = df[5]
df.drop(columns=[0,1,2,3,4,5], inplace=True)
df.dtypes

year             int64
pick_number      int64
team            object
player_name     object
bonus          float64
career_war     float64
dtype: object

In [9]:
df[df['bonus'].isna()]

,year,pick_number,team,player_name,bonus,career_war
228,1997,23,Expos,Donnie Bridges,NaN,NaN
231,1997,37,Expos,*Chris Stowe,NaN,NaN
232,1997,47,Expos,*T.J. Tucker,NaN,0.6
233,1997,48,Expos,*Shane Arthurs,NaN,NaN
241,1996,34,Red Sox,*Chris Reitsma,NaN,0.6
242,1996,35,Braves,*Jason Marquis,NaN,6.8
248,1995,30,Phillies,*Dave Coggin,NaN,0.2
260,1993,41,Blue Jays,*Jeremy Lee,NaN,NaN
268,1991,28,Cardinals,*Tom McKinnon,NaN,NaN
269,1991,30,Royals,*Jason Pruitt,NaN,NaN


In [10]:
df['team'].value_counts()
# team-franchise mapping out of scope for now

team
Braves                   24
Padres                   19
Phillies                 19
Red Sox                  18
Tigers                   17
                         ..
White Sox via Yankees     1
Reds via Yankees          1
Cubs via Reds             1
Giants via Angels         1
Athletics via Rangers     1
Name: count, Length: 74, dtype: int64

In [11]:
df['team_clean'] = df['team'].apply(lambda x: x.partition('via')[0].rstrip())
df['player_name_clean'] = df['player_name'].apply(lambda x: x.removeprefix('*'))
df['bonus_int'] = df['bonus'].fillna(0).astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   year               418 non-null    int64  
 1   pick_number        418 non-null    int64  
 2   team               418 non-null    object 
 3   player_name        418 non-null    object 
 4   bonus              399 non-null    float64
 5   career_war         252 non-null    float64
 6   team_clean         418 non-null    object 
 7   player_name_clean  418 non-null    object 
 8   bonus_int          418 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 29.5+ KB


In [12]:
df_cleaned = pd.DataFrame()
df_cleaned['year'] = df['year']
df_cleaned['pick_number'] = df['pick_number']
df_cleaned['team'] = df['team_clean']
df_cleaned['player_name'] = df['player_name_clean']
df_cleaned['bonus'] = df['bonus_int']
df_cleaned['career_war'] = df['career_war']
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         418 non-null    int64  
 1   pick_number  418 non-null    int64  
 2   team         418 non-null    object 
 3   player_name  418 non-null    object 
 4   bonus        418 non-null    int64  
 5   career_war   252 non-null    float64
dtypes: float64(1), int64(3), object(2)
memory usage: 19.7+ KB


In [13]:
clean_table_name = 'Clean'
sql_create_cleaned_table = f"""
    CREATE TABLE {clean_table_name} (
        year INTEGER NOT NULL,
        pick_number INTEGER NOT NULL,
        team TEXT NOT NULL,
        player_name TEXT NOT NULL,
        bonus INTEGER NOT NULL,
        career_war REAL
    );
"""
sql.query_db(sql_create_cleaned_table, DB_FILEPATH)
sql.query_db(f"PRAGMA table_info({clean_table_name})", DB_FILEPATH)

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Query returned 6 results.


[(0, 'year', 'INTEGER', 1, None, 0),
 (1, 'pick_number', 'INTEGER', 1, None, 0),
 (2, 'team', 'TEXT', 1, None, 0),
 (3, 'player_name', 'TEXT', 1, None, 0),
 (4, 'bonus', 'INTEGER', 1, None, 0),
 (5, 'career_war', 'REAL', 0, None, 0)]

In [14]:
sql.load_pandas_to_table(df_cleaned, DB_FILEPATH, clean_table_name)

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Added 418 entries to 'Clean'.


In [15]:
sql.query_db(f"SELECT MAX(rowid) FROM {clean_table_name}", DB_FILEPATH)

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Query returned 1 results.


[(418,)]

In [16]:
sql_get_n_picks_by_year = f"SELECT year, COUNT(rowid) FROM {clean_table_name} GROUP BY year;"
n_picks_by_year = pd.DataFrame(sql.query_db(sql_get_n_picks_by_year, DB_FILEPATH), columns=['year', 'n_picks'])
n_picks_by_year.set_index('year', inplace=True)
n_picks_by_year.info()

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Query returned 61 results.
<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 1965 to 2025
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   n_picks  61 non-null     int64
dtypes: int64(1)
memory usage: 976.0 bytes


In [17]:
sql_get_mlb_by_year = f"""
    SELECT
        year,
        AVG(bonus),
        COUNT(career_war) AS n_made_mlb,
        AVG(career_war)
    FROM
        {clean_table_name}
    GROUP BY
        year;
"""
cols = ['year', 'avg_bonus', 'n_made_mlb', 'avg_war']
mlb_by_year = pd.DataFrame(sql.query_db(sql_get_mlb_by_year, DB_FILEPATH), columns=cols)
mlb_by_year = n_picks_by_year.join(mlb_by_year.set_index('year')) # SQL join in .py refactor
mlb_by_year.info()

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Query returned 61 results.
<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, 1965 to 2025
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   n_picks     61 non-null     int64  
 1   avg_bonus   61 non-null     float64
 2   n_made_mlb  61 non-null     int64  
 3   avg_war     57 non-null     float64
dtypes: float64(2), int64(2)
memory usage: 2.4 KB


In [18]:
mlb_by_year[mlb_by_year['avg_war'].isna()].n_made_mlb.value_counts()

n_made_mlb
0    4
Name: count, dtype: int64

In [19]:
sql_create_mlb_table = """
    CREATE TABLE MLBByYear (
        year INTEGER NOT NULL PRIMARY KEY,
        n_picks INTEGER NOT NULL,
        avg_bonus INTEGER NOT NULL,
        n_made_mlb INTEGER NOT NULL,
        avg_war REAL
    ) WITHOUT ROWID;
"""
sql.query_db(sql_create_mlb_table, DB_FILEPATH)
sql.query_db("PRAGMA table_info(MLBByYear)", DB_FILEPATH)

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Query returned 5 results.


[(0, 'year', 'INTEGER', 1, None, 1),
 (1, 'n_picks', 'INTEGER', 1, None, 0),
 (2, 'avg_bonus', 'INTEGER', 1, None, 0),
 (3, 'n_made_mlb', 'INTEGER', 1, None, 0),
 (4, 'avg_war', 'REAL', 0, None, 0)]

In [20]:
sql.load_pandas_to_table(mlb_by_year.reset_index(), DB_FILEPATH, 'MLBByYear')

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Added 61 entries to 'MLBByYear'.


In [21]:
sql_get_teams_by_year = f"""
    SELECT
        team,
        year,
        COUNT(rowid),
        AVG(bonus),
        COUNT(career_war) AS n_made_mlb,
        AVG(career_war)
    FROM
        {clean_table_name}
    GROUP BY
        team,
        year;
"""
cols = ['team', 'year', 'n_picks', 'avg_bonus', 'n_made_mlb', 'avg_war']
teams_by_year = pd.DataFrame(sql.query_db(sql_get_teams_by_year, DB_FILEPATH), columns=cols)
teams_by_year.set_index(['team', 'year'], inplace=True)
teams_by_year.info()

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Query returned 377 results.
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 377 entries, ('Angels', np.int64(1968)) to ('Yankees', np.int64(2013))
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   n_picks     377 non-null    int64  
 1   avg_bonus   377 non-null    float64
 2   n_made_mlb  377 non-null    int64  
 3   avg_war     238 non-null    float64
dtypes: float64(2), int64(2)
memory usage: 16.4+ KB


In [22]:
teams_by_year[teams_by_year['avg_war'].isna()].n_made_mlb.value_counts()

n_made_mlb
0    139
Name: count, dtype: int64

In [23]:
sql_create_team_table = """
    CREATE TABLE TeamsByYear (
        team TEXT NOT NULL,
        year INTEGER NOT NULL,
        n_picks INTEGER NOT NULL,
        avg_bonus INTEGER NOT NULL,
        n_made_mlb INTEGER NOT NULL,
        avg_war REAL
    );
"""
sql.query_db(sql_create_team_table, DB_FILEPATH)
sql.query_db("PRAGMA table_info(TeamsByYear)", DB_FILEPATH)

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Query returned 6 results.


[(0, 'team', 'TEXT', 1, None, 0),
 (1, 'year', 'INTEGER', 1, None, 0),
 (2, 'n_picks', 'INTEGER', 1, None, 0),
 (3, 'avg_bonus', 'INTEGER', 1, None, 0),
 (4, 'n_made_mlb', 'INTEGER', 1, None, 0),
 (5, 'avg_war', 'REAL', 0, None, 0)]

In [24]:
sql.load_pandas_to_table(teams_by_year.reset_index(), DB_FILEPATH, 'TeamsByYear')

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Added 377 entries to 'TeamsByYear'.


In [25]:
sql_get_rolling_3_year = """
    SELECT
        year,
        MIN(year) OVER (
            ORDER BY year
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) AS begin_year,
        SUM(n_picks) OVER (
            ORDER BY year
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ),
        AVG(avg_bonus) OVER (
            ORDER BY year
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ),
        SUM(n_made_mlb) OVER (
            ORDER BY year
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ),
        AVG(avg_war) OVER (
            ORDER BY year
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        )
    FROM
        MLBByYear
    """
cols = ['year', 'begin_year', 'n_picks', 'avg_bonus', 'n_made_mlb', 'avg_war']
rolling_3_year = pd.DataFrame(sql.query_db(sql_get_rolling_3_year, DB_FILEPATH), columns=cols)
rolling_3_year

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Query returned 61 results.


,year,begin_year,n_picks,avg_bonus,n_made_mlb,avg_war
0,1965,1965,5,4.400000e+04,4,5.325000
1,1966,1965,13,4.237500e+04,9,7.402500
2,1967,1965,19,4.291667e+04,13,8.268333
3,1968,1966,17,4.269444e+04,11,6.176667
4,1969,1967,16,4.003016e+04,12,5.816667
...,...,...,...,...,...,...
56,2021,2019,13,3.117673e+06,5,0.216667
57,2022,2020,14,3.291994e+06,3,-0.150000
58,2023,2021,15,3.373778e+06,2,-0.100000
59,2024,2022,14,3.110278e+06,0,NaN


In [26]:
sql_get_teams_rolling_3_year = """
    SELECT
        team,
        year,
        MIN(year) OVER (
            PARTITION BY team
            ORDER BY year
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) AS begin_year,
        SUM(n_picks) OVER (
            PARTITION BY team
            ORDER BY year
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ),
        AVG(avg_bonus) OVER (
            PARTITION BY team
            ORDER BY year
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ),
        SUM(n_made_mlb) OVER (
            PARTITION BY team
            ORDER BY year
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ),
        AVG(avg_war) OVER (
            PARTITION BY team
            ORDER BY year
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        )
    FROM
        TeamsByYear
    """
cols = ['team', 'year', 'begin_year', 'n_picks', 'avg_bonus', 'n_made_mlb', 'avg_war']
teams_rolling_3_year = pd.DataFrame(sql.query_db(sql_get_teams_rolling_3_year, DB_FILEPATH), columns=cols)
teams_rolling_3_year

Connected to SQLite database at '../data/sqlite/round_1_hs_pitchers.db'.
Query returned 377 results.


,team,year,begin_year,n_picks,avg_bonus,n_made_mlb,avg_war
0,Angels,1968,1968,1,7.500000e+04,1,-1.800000
1,Angels,1971,1968,2,6.250000e+04,2,27.650000
2,Angels,1977,1968,3,6.166667e+04,3,23.733333
3,Angels,1982,1971,3,6.750000e+04,3,23.900000
4,Angels,1986,1977,3,5.083333e+04,2,7.300000
...,...,...,...,...,...,...,...
372,Yankees,1993,1985,3,7.233333e+05,0,NaN
373,Yankees,2004,1991,3,1.190000e+06,1,11.000000
374,Yankees,2010,1993,3,9.913333e+05,1,11.000000
375,Yankees,2012,2004,3,1.184667e+06,1,11.000000


Buggy. Need to modify TeamsByYear table to include NULL entries for years in which a team did not draft a high school pitcher or pull data directly from Clean table

May consider basing aggregations on n_picks (current implementation) with mlb annual averages as a reference point for an additional metric